# 7. Visualização Cartográfica

"A criação de mapas é uma das habilidades intelectuais mais antigas da humanidade e também uma das mais complexas, com teoria científica, representação gráfica, fatos geográficos e considerações práticas misturadas de uma variedade interminável de maneiras."  &mdash; [H. J. Steward](https://books.google.com/books?id=cVy1Ms43fFYC)

Cartografia &ndash; o estudo e prática de criação de mapas &ndash; possui uma rica história que abrange séculos de descoberta e _design_. A visualização cartográfica aproveita técnicas de mapeamento para transmitir dados contendo informações espaciais, como locais, rotas ou trajetórias na superfície da Terra.

<div style="float: right; margin-left: 1em; margin-top: 1em;"><img width="300px" src="https://gist.githubusercontent.com/jheer/c90d582ef5322582cf4960ec7689f6f6/raw/8dc92382a837ccc34c076f4ce7dd864e7893324a/latlon.png" /></div>

Aproximando a Terra a uma esfera, podemos denotar posições utilizando um sistema de coordenadas esféricas de _latitude_ (ângulo em graus ao sul ou norte da linha do equador) e _longitude_ (angulo em graus que especifica a direção leste-oeste). Nesse sistema, um paralelo é um círculo de latitude constante e um meridiano é um círculo de longitude constante. O  [meridiano principal](https://en.wikipedia.org/wiki/Prime_meridian) se encontra  a 0° de longitude e, por convenção, é definido como passando pelo Observatório Real em Greenwich, Inglaterra.

Para "planificar" uma esfera tridimensional, em um plano bidimensional, precisamos aplicar uma [projeção](https://en.wikipedia.org/wiki/Map_projection) que mapeia (`longitude`,  `latitude`) em pares de coordenadas (`x`,`y`). Semelhante a [escalas](https://github.com/uwdata/visualization-curriculum/blob/master/altair_scales_axes_legends.ipynb), projeções mapeiam de um domínio de dados (posição espacial) para um intervalo visual (posição de pixel). No entanto, os mapeamentos de escala que vimos até agora aceitam um domínio unidimensional, enquanto projeções de mapas são inerentemente bidimensionais.

Nesse _notebook_, introduziremos o básico da criação de mapas e visualização espacial com o _Altair_, incluindo:

- Formas de dados para representar recursos geográficos,
- Técnicas de Geovisualização como ponto, símbolo, e mapas cloropléticos, e
- Uma revisão de projeções cartográficas comuns.

Esse notebook é parte do [currículo de vizualização de dados](https://github.com/uwdata/visualization-curriculum).

In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data

## 7.1. Dados geográficos: GeoJSON e TopoJSON

Até esse ponto, já trabalhamos previamente bases de dados formatadas em JSON e CSV, que correspondem a tabelas de dados feitas de linhas (registros) e colunas (campos). A fim de representar regiões geográficas (países, estados, _etc._) e trajetórias (rotas de voo, linhas de metrô, _etc._), precisamos expandir nosso repertório com formatos adicionais projetados para suportar geometrias mais complexas.

[GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) modela características geográficas com um formato JSON especializado. Uma `feature` GeoJSON pode incluir dados geométricos &ndash; como coordenadas `longitude`, `latitude` que criam os limites de um país &ndash; bem como atributos de dados adicionais.

Aqui está um objeto `feature` GeoJSON para a fronteira do estado do Colorado nos Estados Unidos:

~~~ json
{
  "type": "Feature",
  "id": 8,
  "properties": {"name": "Colorado"},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [[-106.32056285448942,40.998675790862656],[-106.19134826714341,40.99813863734313],[-105.27607827344248,40.99813863734313],[-104.9422739227986,40.99813863734313],[-104.05212898774828,41.00136155846029],[-103.57475287338661,41.00189871197981],[-103.38093099236758,41.00189871197981],[-102.65589358559272,41.00189871197981],[-102.62000064466328,41.00189871197981],[-102.052892177978,41.00189871197981],[-102.052892177978,40.74889940428302],[-102.052892177978,40.69733266640851],[-102.052892177978,40.44003613055551],[-102.052892177978,40.3492571857556],[-102.052892177978,40.00333031918079],[-102.04930288388505,39.57414465707943],[-102.04930288388505,39.56823596836465],[-102.0457135897921,39.1331416175485],[-102.0457135897921,39.0466599009048],[-102.0457135897921,38.69751011321283],[-102.0457135897921,38.61478847120581],[-102.0457135897921,38.268861604631],[-102.0457135897921,38.262415762396685],[-102.04212429569915,37.738153927339205],[-102.04212429569915,37.64415206142214],[-102.04212429569915,37.38900413964724],[-102.04212429569915,36.99365914927603],[-103.00046581851544,37.00010499151034],[-103.08660887674611,37.00010499151034],[-104.00905745863294,36.99580776335414],[-105.15404227428235,36.995270609834606],[-105.2222388620483,36.995270609834606],[-105.7175614468747,36.99580776335414],[-106.00829426840322,36.995270609834606],[-106.47490250048605,36.99365914927603],[-107.4224761410235,37.00010499151034],[-107.48349414060355,37.00010499151034],[-108.38081766383978,36.99903068447129],[-109.04483707103458,36.99903068447129],[-109.04483707103458,37.484617466122884],[-109.04124777694163,37.88049961001363],[-109.04124777694163,38.15283644441336],[-109.05919424740635,38.49983761802722],[-109.05201565922046,39.36680339854235],[-109.05201565922046,39.49786885730673],[-109.05201565922046,39.66062637372313],[-109.05201565922046,40.22248895514744],[-109.05201565922046,40.653823231326896],[-109.05201565922046,41.000287251421234],[-107.91779872584989,41.00189871197981],[-107.3183866123281,41.00297301901887],[-106.85895696843116,41.00189871197981],[-106.32056285448942,40.998675790862656]]
    ]
  }
}
~~~

A `feature` inclui um objeto `properties`, o qual pode incluir qualquer número de campos de dados, junto ao objeto `geometry`,o qual, nesse caso, contém um simples polígono que consiste nas coordenadas `[longitude, latitude]`  para a fronteira do estado. As coordenadas continuam para a direita por um tempo, caso você queira rolar...

Para saber mais sobre os detalhes essenciais do GeoJSON, veja [especificação oficial do GeoJSON](http://geojson.org/) ou leia [Tom MacWright's helpful primer](https://macwright.org/2015/03/23/geojson-second-bite).

Uma desvantagem do GeoJSON como formato de armazenamento é que ele pode ser redundante, resultando em tamanhos de arquivo maiores. Considere: o Colorado compartilha fronteiras com seis outros estados (sete se você incluir o canto que toca o Arizona). Em vez de usar listas separadas, sobrepostas de coordenadas para cada um desses estados, uma abordagem mais compacta é codificar bordas compartilhadas apenas uma vez, representando a _topologia_ de regiões geográficas. Felizmente, isso é precisamente o que o formato [TopoJSON](https://github.com/topojson/topojson/blob/master/README.md) faz!

Vamos carregar um arquivo TopoJSON dos países do mundo (com resolução de 110 metros):

In [2]:
world = data.world_110m.url
world

'https://vega.github.io/vega-datasets/data/world-110m.json'

In [3]:
world_topo = data.world_110m()

In [4]:
world_topo.keys()

dict_keys(['type', 'transform', 'objects', 'arcs'])

In [5]:
world_topo['type']

'Topology'

In [6]:
world_topo['objects'].keys()

dict_keys(['land', 'countries'])

_Inspecione o objeto-dicionário TopoJSON `world_topo` acima para ver seus conteúdos._

Nos dados acima, a propriedade `objects` indica os elementos nomeados que podemos extrair dos dados: geometrias para todos os países (`countries`), ou um simples polígono representando todas as terras (`land`) no planeta Terra. Qualquer um deles pode ser descompactado em dados GeoJSON que podemos visualizar.

Como TopoJSON é um formato especializado, precisamos instruir o _Altair_ para analisar o formato TopoJSON, indicando qual objeto de recurso nomeado desejamos extrair da topologia. O código a seguir indica que queremos extrair recursos GeoJSON do _dataset_ `world` para o objeto `countries`:

~~~ js
alt.topo_feature(world, 'countries')
~~~

Esse método `alt.topo_feature` expande para o seguinte Vega-Lite JSON:

~~~ json
{
  "values": world,
  "format": {"type": "topojson", "feature": "countries"}
}
~~~

Agora que podemos carregar dados geográficos, estamos prontos para começar a fazer mapas!

## 7.2 Marcas de Formas Geográficas

Para visualizar dados geográficos, o _Altair_ possui a marca do tipo `geoshape`. Para criar um mapa básico, podemos criar um marcador `geoshape` e passá-lo os nossos dados TopoJSON, que são então descompactados em recursos GeoJSON, um para cada país do mundo:

In [7]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape()

alt.Chart(...)

No exemplo acima, _Altair_ aplica uma cor azul padrão e utiliza uma projeção de mapa (`mercator`) padrão. Podemos customizar as cores e as larguras dos traçados das fronteiras utilizando propriedades de marca padrões. Usando o método `project` podemos também adicionar nossa própria projeção de mapa:

In [8]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator'
)

alt.Chart(...)

Por padrão, o _Altair_ automaticamente ajusta a projeção para que todos os dados caibam na largura e altura do gráfico. Também podemos especificar parâmetros de projeção, como `scale` (nível de zoom) e `translate` (panorâmica), para personalizar as configurações de projeção. Aqui, ajustamos os parâmetros `scale` e `translate` para focar na Europa:

In [9]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator', scale=400, translate=[100, 550]
)

alt.Chart(...)

_Note como a resolução de 110m dos dados se torna aparente nessa escala. Para ver linhas e limites costeiros mais detalhados, precisamos de um arquivo de entrada com geometrias mais refinadas. Ajuste os parâmetros `scale` e  `translate` para focar o mapa em outras regiões!_

Até agora, nosso mapa mostra apenas países. Usando o operador `layer`, podemos combinar múltiplos elementos de mapa. O _Altair_ inclui _data generators_ (geradores de dados) que podemos usar para criar dados para camadas de mapa adicionais:

- O gerador de esfera (`{'sphere': True}`) produz uma representação GeoJSON da esfera completa da Terra. Podemos criar uma marca adicional `geoshape` que preenche a forma da Terra como uma camada de fundo.
-  O gerador de gratícula (`{'graticule': ...}`) cria um recurso GeoJSON representando uma graticula: uma grade formada por linhas de latitude e longitude. A gratícula padrão tem meridianos e paralelos a cada 10° entre ±80° de latitude. Para as regiões polares, existem meridianos a cada 90°. Essas configurações podem ser personalizadas usando as propriedades `stepMinor` e`stepMajor`.

Vamos colocar em camadas marcas de esfera, gratícula e país em uma especificação de mapa reutilizável:

In [ ]:
map = alt.layer(
    # Usar a esfera da Terra como a camada base
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # Adicionar uma gratícula para linhas de referência geográfica
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # e então os países do mundo
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
    )
).properties(
    width=600,
    height=400
)

Podemos estender o mapa com uma projeção desejada e desenhar o resultado. Aqui aplicamos uma [Projeção natural da Terra](https://en.wikipedia.org/wiki/Natural_Earth_projection). A camada _sphere_ (esfera) produz o fundo azul claro; a camada _graticule_ (gratícula) produz as linhas brancas de referência geográfica.

In [11]:
map.project(
    type='naturalEarth1', scale=110, translate=[300, 200]
).configure_view(stroke=None)

alt.LayerChart(...)

## 7.3. Mapas de Pontos

Além dos dados *geométricos* fornecidos por arquivos GeoJSON ou TopoJSON, muitos conjuntos de dados tabulares incluem informações geográficas na forma de campos para coordenadas de `longitude` e `latitude`, ou referências a regiões geográficas como nomes de países, nomes de estados, códigos postais, _etc._, que podem ser mapeadas em coordenadas usando um [serviço de geocodificação](https://en.wikipedia.org/wiki/Geocoding). Em alguns casos, os dados de localização são ricos o suficiente para que possamos observar padrões significativos apenas projetando os pontos de dados &mdash; sem necessidade de mapa base!

Vamos analisar um _dataset_ de códigos postais de 5 dígitos nos Estados Unidos, incluindo coordenadas de `longitude` e `latitude` para cada agência, além de um campo `zip_code`.

In [12]:
zipcodes = data.zipcodes.url
zipcodes

'https://vega.github.io/vega-datasets/data/zipcodes.csv'

Podemos visualizar a localização de cada agência usando uma pequena marca `square` (quadrada, de 1 pixel). No entanto, para definir as posições, _não_ usamos os canais `x` e `y`. _Por quê?_

Embora projeções cartográficas mapeiem coordenadas (`longitude`, `latitude`) para as coordenadas (`x`, `y`), elas podem fazer isso de maneiras arbitrárias. Não há garantia, por exemplo, de que `longitude` → `x` e `latitude` → `y`! Em vez disso, o Altair inclui canais especiais de codificação de `longitude` e `latitude` para lidar com coordenadas geográficas. Esses canais indicam quais campos de dados devem ser mapeados para coordenadas de `longitude` e `latitude`, e então aplica-se uma projeção para mapear essas coordenadas para posições (`x`, `y`).

In [ ]:
alt.Chart(zipcodes).mark_square(
    size=1, opacity=1
).encode(
    longitude='longitude:Q', # aplica o campo 'longitude' ao canal longitude
    latitude='latitude:Q'    # aplica o campo 'latitude' ao canal latitude
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Ao plotar apenas os códigos postais, podemos ver o contorno dos Estados Unidos e discernir padrões significativos na densidade das agências, sem um mapa base ou elementos de referência adicionais!_

Usamos a projeção `albersUsa`, que toma algumas liberdades com a geometria real da Terra, com versões escaladas do Alasca e do Havai no canto inferior esquerdo. Como não especificamos parâmetros de projeção `scale` ou `translate`, o Altair os ajusta automaticamente para se adequar aos dados visualizados.

Agora podemos fazer mais perguntas sobre nosso _dataset_. Por exemplo, há alguma lógica ou razão na alocação dos códigos postais? Para avaliar essa questão, podemos adicionar uma codificação de cor baseada no primeiro dígito do código postal. Primeiro, adicionamos uma transformação `calculate` para extrair o primeiro dígito, e codificamos o resultado usando o canal de cor:

In [14]:
alt.Chart(zipcodes).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=2, opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Para dar zoom em um dígito específico, adicione uma transformação de filtro (filter transform) para limitar os dados exibidos! Tente adicionar uma [seleção interativa ](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb) para filtrar para um único dígito e atualizar dinamicamente o mapa. E tenha certifique-se de usar strings (\`'1'\`) em vez de números (\`1\`) ao filtrar os valores dos dígitos!_

(Este exemplo é inspirado na clássica visualização [zipdecode](https://benfry.com/zipdecode/) de Ben Fry!)

Podemos nos perguntar ainda o que a _sequência_ dos códigos postais pode indicar. Uma maneira de explorar essa questão é conectar cada código postal consecutivo usando uma marca `line`, como feito na visualização [ZipScribble](https://eagereyes.org/zipscribble-maps/united-states) de Robert Kosara:

In [15]:
alt.Chart(zipcodes).transform_filter(
    '-150 < datum.longitude && 22 < datum.latitude && datum.latitude < 55'
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_line(
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N',
    order='zip_code:O'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Podemos ver agora como os códigos postais se agrupam ainda mais em áreas menores, indicando uma alocação hierárquica dos códigos por localização, mas com alguma variabilidade notável dentro dos clusters locais._

Se você estava prestando atenção aos nossos mapas anteriores, pode ter notado que há códigos postais sendo plotados no canto superior esquerdo! Estes correspondem a locais como Porto Rico ou Samoa Americana, que contêm códigos postais dos EUA, mas são mapeados para coordenadas `null` (nulas) (`0`, `0`) pela projeção `albersUsa`. Além disso, o Alasca e o Havai podem complicar nossa visão dos segmentos de reta conectando os pontos. Em resposta, o código acima inclui um filtro adicional que remove pontos fora dos nossos intervalos escolhidos de `longitude` e `latitude`.

_Remova o filtro acima para ver o que acontece!_

## 7.4. Mapas de Símbolo

Agora, vamos combinar um mapa base e dados plotados como camadas separadas. Nós examinaremos a rede de voos comerciais dos Estados Unidos, considerando tanto aeroportos quanto rotas de voo. Para fazer isso, precisaremos de três conjuntos de dados.
Para o nosso mapa base, usaremos um arquivo TopoJSON dos Estados Unidos com resolução de 10m, contendo recursos para `states` (estados) ou `counties` (condados):

In [ ]:
usa = data.us_10m.url
usa

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json'

Para os aeroportos, usaremos um dataset com campos para as coordenadas de `longitude` e `latitude` de cada aeroporto, bem como o código `iata` do aeroporto &mdash; por exemplo, `'SEA'` para o [Aeroporto Internacional de Seattle-Tacoma](https://en.wikipedia.org/wiki/Seattle%E2%80%93Tacoma_International_Airport).

In [ ]:
airports = data.airports.url
airports

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/airports.csv'

Por fim, usaremos um _dataset_ de rotas de voo, que contém campos `origin` (origem) e `destination` (destino) com os códigos IATA dos aeroportos correspondentes.

In [ ]:
flights = data.flights_airport.url
flights

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/flights-airport.csv'

Vamos começar criando um mapa base usando a projeção `albersUsa`, e adicionar uma camada que plota marcas `cricle` para cada aeroporto.

In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_Isso é um monte de aeroportos! Obviamente, nem todos são grandes hubs._

Assim como nosso _dataset_ de códigos postais, nossos dados de aeroportos incluem pontos que ficam fora dos Estados Unidos continental. Por isso, nós novamente vemos pontos no canto superior esquerdo. Podemos querer filtrar esses pontos, mas, para isso, primeiro precisamos saber mais sobre eles.

_Atualize a projeção do mapa acima para `albers` &ndash; evitando o comportamento idiossincrático do `albersUsa` &ndash; para que as localizações reais desses pontos adicionais sejam reveladam!_

Agora, em vez de mostrar todos os aeroportos de forma indiferenciada, vamos identificar os principais _hubs_ considerando o número total de rotas que se originam em cada aeroporto. Usaremos o _dataset_ `routes` (rotas) como nossa fonte de dados primária: ele contém uma lista de rotas de voo que podemos agregar para contar o número de rotas para cada aeroporto `origin` (de origem).

No entanto, o _dataset_ `routes` não inclui as _localizações_ dos aeroportos! Para incrementar os dados de `routes` com localizações, precisamos de uma nova transformação de dados: `lookup`. A transformação `lookup` pega um valor de campo em um _dataset_ primário e usa-o como uma _key_ (chave) para procurar informações relacionadas em outra tabela. Neste caso, queremos corresponder o código `origin` do aeroporto em nosso _dataset_ `routes` com o campo `iata` do _dataset_ `airports` e, em seguida, extrair os campos de `latitude` e `longitude` correspondentes.

In [ ]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

_Quais aeroportos dos EUA têm o maior número de rotas de saída?_

Agora que podemos ver os aeroportos, podemos querer interagir com eles para entender melhor a estrutura da rede de tráfego aéreo. Podemos adicionar uma camada de marcas `rule` para representar caminhos de aeroportos `origin` para aeroportos `destination`, o que requer duas transformações `lookup` para recuperar as coordenadas de cada ponto final. Além disso, podemos usar uma seleção `single` para filtrar essas rotas, de modo que apenas rotas que se originam no aeroporto selecionado no momento sejam mostradas.

_Começando a partir do mapa estático acima, você consegue construir uma versão interativa? Sinta-se à vontade para pular o código abaixo e interagir primeiro com o mapa interativo, e depois pensar em como você poderia construí-lo por conta própria!_

In [ ]:
# seleção interativa para o aeroporto de origem
# seleciona o aeroporto mais próximo ao cursor do mouse
origin = alt.selection_single(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)

# referência de dados compartilhada para transformações de lookup
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])

alt.layer(
    # mapa base dos Estados Unidos
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # linhas de rota do aeroporto de origem selecionado para os aeroportos de destino
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filtrar apenas para a origem selecionada
    ).transform_lookup(
        lookup='origin', from_=foreign # lookup para latitude/longitude de origem
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # lookup para latitude/longitude de destino, renomeando os campos
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # dimensionar aeroportos pelo número de rotas de saída
    # 1. agregar o dataset de voos-aeroportos
    # 2. buscar dados de localização do dataset de aeroportos
    # 3. remover Porto Rico (PR) e Ilhas Virgens (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_selection(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # ordenar círculos menores por cima
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None # remover bordas da visualização
)

<ipython-input-9-c7c64ca18a12>:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  origin = alt.selection_single(
<ipython-input-9-c7c64ca18a12>:45: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.LayerChart(...)

_Passe o mouse sobre o mapa para explorar a rede de voos!_



## 7.5. Mapas Coropléticos

Um [mapa coroplético](https://en.wikipedia.org/wiki/Choropleth_map) usa regiões sombreadas ou texturizadas para visualizar valores de dados. Mapas com símbolos dimensionados são frequentemente mais precisos para leitura, pois as pessoas tendem a ser melhores em estimar diferenças proporcionais entre a área de círculos do que entre tons de cores. No entanto, os mapas coropléticos são populares na prática e particularmente úteis quando muitos símbolos se tornam perceptualmente sobrecarregados.

Por exemplo, enquanto os Estados Unidos têm apenas 50 estados, eles têm milhares de condados dentro desses estados. Vamos construir um mapa coroplético da taxa de desemprego por condado, no ano de recessão de 2008. Em alguns casos, arquivos GeoJSON ou TopoJSON de entrada podem incluir dados estatísticos que podemos visualizar diretamente. Neste caso, no entanto, temos dois arquivos: nosso arquivo TopoJSON que inclui os limites dos condados (`usa`) e um arquivo de texto separado que contém estatísticas de desemprego:

In [ ]:
unemp = data.unemployment.url
unemp

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/unemployment.tsv'

Para integrar nossas fontes de dados, precisaremos novamente usar a transformação `lookup`, enriquecendo nossos dados de `geoshape` baseados em TopoJSON com as taxas de desemprego. Podemos então criar um mapa que inclua um campo de codificação `color` para o campo `rate` procurado.

In [ ]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).encode(
    alt.Color('rate:Q',
              scale=alt.Scale(domain=[0, 0.3], clamp=True),
              legend=alt.Legend(format='%')),
    alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*Examine as taxas de desemprego por condado. Valores mais altos em Michigan podem estar relacionados à indústria automotiva. Condados nos estados das [Grandes Planícies](https://en.wikipedia.org/wiki/Great_Plains) e Montanhas exibem taxas tanto baixas **quanto** altas. Essa variação é significativa ou possivelmente um [artefato de tamanhos de amostra menores](https://medium.com/@uwdata/surprise-maps-showing-the-unexpected-e92b67398865)? Para explorar mais, tente alterar o domínio superior da escala (por exemplo, para `0,2`) para ajustar o mapeamento de cores.*

Uma preocupação central para mapas coropléticos é a escolha de cores. Acima, usamos o esquema padrão `'yellowgreenblue'` do Altair para mapas de calor. Abaixo, comparamos outros esquemas, incluindo um esquema _sequencial de tom único_ (`teals`) que varia apenas em luminância, um esquema _sequencial de múltiplos tons_ (`viridis`) que varia em luminância e matiz, e um esquema _divergente_ (`blueorange`) que usa um ponto médio branco:

In [ ]:
# função de utilidade para gerar uma especficação de mapa para um esquema de cores fornecido
def map_(scheme):
    return alt.Chart().mark_geoshape().project(type='albersUsa').encode(
        alt.Color('rate:Q', scale=alt.Scale(scheme=scheme), legend=None)
    ).properties(width=305, height=200)

alt.hconcat(
    map_('tealblues'), map_('viridis'), map_('blueorange'),
    data=alt.topo_feature(usa, 'counties')
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).configure_view(
    stroke=None
).resolve_scale(
    color='independent'
)

alt.HConcatChart(...)

_Quais esquemas de cores você considera mais eficazes? Por quê? Modifique os mapas acima para usar outros esquemas disponíveis, conforme descrito na [documentação dos Esquemas de Cores do Vega](https://vega.github.io/vega/docs/schemes/)._

## 7.6. Projeções Cartográficas

Agora que temos alguma experiência na criação de mapas, vamos examinar mais de perto as projeções cartográficas. Como explicado pela [Wikipedia](https://en.wikipedia.org/wiki/Map_projection),

> _Todas as projeções de mapas necessariamente distorcem a superfície de alguma forma. Dependendo do propósito do mapa, algumas distorções são aceitáveis e outras não; portanto, diferentes projeções existem para preservar algumas propriedades do corpo esférico à custa de outras propriedades._

Algumas das propriedades que podemos querer considerar incluem:

- _Área_: A projeção distorce o tamanho das regiões?
- _Rumo_: Uma linha reta corresponde a uma direção de viagem constante?
- _Distância_: Linhas de mesmo comprimento correspondem a distâncias iguais no globo?
- _Forma_: A projeção preserva as relações espaciais (ângulos) entre os pontos?

A escolha da projeção apropriada depende, portanto, do caso de uso do mapa. Por exemplo, se estivermos avaliando o uso do solo e a extensão das terras for um fator relevante, podemos optar por uma projeção que preserve as áreas. Se quisermos visualizar ondas de choque resultantes de um terremoto, podemos centralizar o mapa no epicentro e preservar as distâncias a partir desse ponto. Ou, se desejarmos providenciar auxílio para navegação, a preservação do rumo e da forma pode ser mais importante.

Também podemos caracterizar as projeções com base na _superfície de projeção_. Projeções cilíndricas, por exemplo, projetam pontos da superfície da esfera em um cilindro ao redor dela; o cilindro "desenrolado" então fornece nosso mapa. Como descrevemos a seguir, podemos alternativamente projetar sobre a superfície de um cone (projeções cônicas) ou diretamente em um plano (projeções azimutais).

*Vamos primeiro construir nossa intuição interagindo com uma variedade de projeções! **[Abra o notebook online de Projeções Cartográficas do Vega-Lite](https://observablehq.com/@vega/vega-lite-cartographic-projections).** Use os controles da página para selecionar uma projeção e explorar os parâmetros de projeção, como `scale` (zoom) e translação x/y (panorâmica). Os controles de rotação ([yaw, pitch, roll](https://en.wikipedia.org/wiki/Aircraft_principal_axes)) determinam a orientação do globo em relação à superfície sobre a qual está sendo projetado.*

### 7.6.1 Um Passeio por Tipos Específicos de Projeção

[**Projeções cilíndricas**](https://en.wikipedia.org/wiki/Map_projection#Cylindrical) mapeiam a esfera em um cilindro circundante, que é então desenrolado. Se o eixo principal do cilindro estiver orientado no sentido norte-sul, os meridianos são mapeados como linhas retas. Projeções [**pseudocilíndricas**](https://en.wikipedia.org/wiki/Map_projection#Pseudocylindrical) representam um meridiano central como uma linha reta, com os demais meridianos "curvando-se" para longe do centro.

In [ ]:
minimap = map.properties(width=225, height=225)
alt.hconcat(
    minimap.project(type='equirectangular').properties(title='equirectangular'),
    minimap.project(type='mercator').properties(title='mercator'),
    minimap.project(type='transverseMercator').properties(title='transverseMercator'),
    minimap.project(type='naturalEarth1').properties(title='naturalEarth1')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Equiretangular](https://en.wikipedia.org/wiki/Equirectangular_projection) (`equirectangular`): Escala os valores de coordenadas `lat`, `lon` diretamente.
- [Mercator](https://en.wikipedia.org/wiki/Mercator_projection) (`mercator`): Projeta em um cilindro, usando `lon` diretamente, mas aplicando uma transformação não linear em `lat`. Linhas retas preservam rumos constantes em bússolas ([linhas loxodrômicas](https://en.wikipedia.org/wiki/Rhumb_line)), tornando essa projeção adequada para a navegação. No entanto, áreas localizadas ao extremo norte ou sul podem ser altamente distorcidas.
- [Mercator Transversa](https://en.wikipedia.org/wiki/Transverse_Mercator_projection) (`transverseMercator`): Uma projeção de mercator, mas com o cilindro de contorno rotacionado para um eixo transversal. Enquanto a projeção de Mercator padrão é mais precisa ao longo do equador, a Mercator Transversa tem maior precisão ao longo do meridiano central.
- [Natural Earth](https://en.wikipedia.org/wiki/Natural_Earth_projection) (`naturalEarth1`): Uma projeção pseudocilíndrica projetada para exibir a Terra inteira em uma única visualização.
<br/><br/>

[**Projeções cônicas**](https://en.wikipedia.org/wiki/Map_projection#Conic) mapeiam a esfera para um cone e então desenrolam o cone sobre o plano. Projeções cônicas são configuradas por dois _paralelos padrão_, que determinam onde o cone intersecta a esfera.

In [ ]:
minimap = map.properties(width=180, height=130)
alt.hconcat(
    minimap.project(type='conicEqualArea').properties(title='conicEqualArea'),
    minimap.project(type='conicEquidistant').properties(title='conicEquidistant'),
    minimap.project(type='conicConformal', scale=35, translate=[90,65]).properties(title='conicConformal'),
    minimap.project(type='albers').properties(title='albers'),
    minimap.project(type='albersUsa').properties(title='albersUsa')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Área Cônica Igual](https://en.wikipedia.org/wiki/Albers_projection) (`conicEqualArea`): Projeção cônica que preserva área. Forma e distância não são preservadas, mas são aproximadamente precisas dentro dos paralelos padrão.
- [Equidistante Cônica](https://en.wikipedia.org/wiki/Equidistant_conic_projection) (`conicEquidistant`): Projeção cônica que preserva distância ao longo dos meridianos e dos paralelos padrão.
- [Cônica Conforme](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) (`conicConformal`): Projeção cônica que preserva forma (ângulos locais), mas não área ou distância.
- [Albers](https://en.wikipedia.org/wiki/Albers_projection) (`albers`): Uma variante da projeção área cônica igual com paralelos padrão otimizados para criar mapas dos Estados Unidos.
- [Albers USA](https://en.wikipedia.org/wiki/Albers_projection) (`albersUsa`): Uma projeção híbrida para os 50 estados dos Estados Unidos da América. Esta projeção junta três projeções Albers com diferentes parâmetros para os Estados Unidos continental, Alasca e Havai.
<br/><br/>

[**Projeções Azimutais**](https://en.wikipedia.org/wiki/Map_projection#Azimuthal_%28projections_onto_a_plane%29) mapeiam a esfera diretamente sobre um plano.

In [ ]:
minimap = map.properties(width=180, height=180)
alt.hconcat(
    minimap.project(type='azimuthalEqualArea').properties(title='azimuthalEqualArea'),
    minimap.project(type='azimuthalEquidistant').properties(title='azimuthalEquidistant'),
    minimap.project(type='orthographic').properties(title='orthographic'),
    minimap.project(type='stereographic').properties(title='stereographic'),
    minimap.project(type='gnomonic').properties(title='gnomonic')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

- [Azimutal Área Igual](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) (`azimuthalEqualArea`): Projeta com precisão áreas em todas as partes do globo, mas não preserva forma (ângulos locais).
- [Azimutal Equidistante](https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection) (`azimuthalEquidistant`): Preserva a distância proporcional do centro da projeção para todos os outros pontos do globo.
- [Ortográfica](https://en.wikipedia.org/wiki/Orthographic_projection_in_cartography) (`orthographic`): Projeta um hemisfério visível sobre um plano distante. Corresponde aproximadamente a uma vista da Terra a partir do espaço sideral.
- [Estereográfica](https://en.wikipedia.org/wiki/Stereographic_projection) (`stereographic`): Preserva forma, mas não área ou distância.
- [Gnômica](https://en.wikipedia.org/wiki/Gnomonic_projection) (`gnomonic`): Projeta a superfície da esfera diretamente sobre um plano tangente. [Círculos máximos](https://en.wikipedia.org/wiki/Great_circle) ao redor da Terra são projetados como linhas retas, mostrando o caminho mais curto entre os pontos.
<br/><br/>

## 7.7. Coda: Manipulando Dados Geográficos

Os exemplos acima foram todos retirados da coleção vega-datasets, incluindo dados geométricos (TopoJSON) e tabulares (aeroportos, taxas de desemprego). Um desafio comum para começar com visualização geográfica é coletar os dados necessários para sua tarefa. Existem diversos provedores de dados, incluindo serviços como o [United States Geological Survey](https://www.usgs.gov/products/data/all-data) e o [U.S. Census Bureau](https://www.census.gov/data/datasets.html).

Em muitos casos você pode ter dados existentes com um componente geográfico, mas precisar de medidas ou geometrias adicionais. Para ajudar você a começar, aqui está um fluxo de trabalho:

1. Visite [Natural Earth Data](http://www.naturalearthdata.com/downloads/) e navegue para selecionar dados para regiões e resoluções de interesse. Baixe o(s) arquivo(s) zip correspondente(s).
2. Acesse [MapShaper](https://mapshaper.org/) e arraste o arquivo zip baixado sobre a página. Revise os dados conforme desejado e, em seguida, "Exporte" os arquivos TopoJSON ou GeoJSON gerados.
3. Carregue os dados exportados do MapShaper para usar com o Altair!

Claro, existem muitas outras ferramentas &ndash; tanto de código aberto quanto proprietárias &ndash; para trabalhar com dados geográficos. Para saber mais sobre manipulação de geo-dados e criação de mapas, veja a série de tutoriais de Mike Bostock sobre [Cartografia via Linha de Comando](https://medium.com/@mbostock/command-line-cartography-part-1-897aa8f8ca2c).

## 7.8 Resumo

Neste ponto, apenas mergulhamos pés nas águas da criação de mapas. _(Você não esperava que um único _notebook_ transmitisse séculos de aprendizado, não é?)_ Por exemplo, deixamos intocados tópicos como [_cartogramas_](https://en.wikipedia.org/wiki/Cartogram) e transmissão de [_topografia_](https://en.wikipedia.org/wiki/Topography) &mdash; como no esclarecedor livro [_Cartographic Relief Presentation_](https://books.google.com/books?id=cVy1Ms43fFYC) de Imhof. No entanto, você deve agora estar bem equipado para criar uma rica variedade de visualizações geográficas. Para mais, o livro de MacEachren [_How Maps Work: Representation, Visualization, and Design_](https://books.google.com/books?id=xhAvN3B0CkUC) fornece uma visão geral valiosa da criação de mapas a partir da perspectiva da visualização de dados.